In [1]:
%pip install -qU langchain langchain-core langchain-community langchain-core langchain-openai pyowm google-search-results

ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install
Note: you may need to restart the kernel to use updated packages.


In [37]:
import os

from langchain_community.utilities import OpenWeatherMapAPIWrapper

os.environ["OPENWEATHERMAP_API_KEY"] = "YOUR_API"

weather = OpenWeatherMapAPIWrapper()

In [38]:
weather_data = weather.run("Michigan, USA")
print(weather_data)

In Michigan, USA, the current weather is as follows:
Detailed status: overcast clouds
Wind speed: 2.57 m/s, direction: 320°
Humidity: 94%
Temperature: 
  - Current: 20.04°C
  - High: 20.04°C
  - Low: 20.04°C
  - Feels like: 20.55°C
Rain: {}
Heat index: None
Cloud cover: 100%


In [9]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32009 sha256=30a1a5555ae834771f4d56661595daebd7b81757acda70f3fe1d5b937ee83955
  Stored in directory: /Users/shubhamrathod/Library/Caches/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [10]:
import os
from langchain_community.utilities import SerpAPIWrapper

In [18]:
os.environ["SERPAPI_API_KEY"] = "YOUR_API"
os.environ["OPENAI_API_KEY"] = 'YOUR_API'

In [15]:
search = SerpAPIWrapper()

In [17]:
search.run("What the current news in michigan?")

'[\'US News provides the latest news involving your state. Explore issues including health care, education, economy, crime and corrections, and infrastructure.\', \'Get the latest Michigan Local News, Sports News & US breaking News. View daily MI weather updates, watch videos and photos, join the discussion in forums.\', \'The latest breaking news, comment and features from The Independent.\', \'Michigan · 5 Michigan residents arrested after methamphetamine seized in separate Port Huron drug busts · 2 killed, 4 hurt in Dearborn crash after driver loses ...\', \'Michigan news headlines and stories from ClickOnDetroit and WDIV Local 4.\', "News, analysis and commentary about the Michigan from the state\'s largest news source, the Detroit Free Press.", \'Michigan gets its latest local and breaking news and analysis and in-depth coverage of state government from The Detroit News.\', \'We tell local Grand Rapids news & weather stories, and we do what we do to make Grand Rapids & the rest of

In [21]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import OpenAI

In [41]:
from langchain.agents import Tool

# You can create the tool to pass to an agent
news_tools = Tool(
    name="news-agent",
    description="Use this tool to get the latest news",
    func=search.run,
)

weather_tools = Tool(
    name="weather-agent",
    description="Use this to answer weather related query",
    func=weather.run,
)

In [42]:
tools = [news_tools, weather_tools]

In [43]:

llm = OpenAI(temperature=0)

# tools = load_tools(news_tools, llm)

agent_chain = initialize_agent(
    tools=tools, 
    llm=llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

In [44]:
agent_chain('Whats the latest news about michigan?')



> Entering new AgentExecutor chain...
 I should use the news-agent tool to get the latest news
Action: news-agent
Action Input: "michigan"
Observation: [{'title': 'Michigan disqualifies presidential candidate Cornel West from November ballot', 'link': 'https://www.freep.com/story/news/politics/elections/2024/08/16/michigan-disqualifies-cornel-west-november-election/74832390007/', 'source': 'Detroit Free Press', 'date': '18 hours ago', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR5Whp8uMp8ME5-044Ka93zpihM3b37U-0hbYguEnWkOg7eivpUZK0e9JQ&usqp=CAI&s'}, {'title': 'There are 10 serene little lakes at this Northern Michigan state park', 'link': 'https://www.mlive.com/life/2024/08/there-are-10-serene-little-lakes-at-this-northern-michigan-state-park.html', 'source': 'MLive.com', 'date': '16 hours ago', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ1Yp3ZdLYSuwCucfJHPKXVwLtIQHMrrk8RYr89brSc67-WRGw_gqO9z34&usqp=CAI&s'}, {'title': 'Witness Michigan 

{'input': 'Whats the latest news about michigan?',
 'output': 'The latest news about Michigan is a mix of political and lifestyle news, including the disqualification of a presidential candidate, a state park with 10 serene lakes, new food options at Michigan Stadium, and a pharmacist pleading no contest in a meningitis outbreak case.'}

In [45]:
agent_chain('Whats is the current weather in michigan?')



> Entering new AgentExecutor chain...
 I should use the weather-agent tool to get the current weather
Action: weather-agent
Action Input: michigan
Observation: In michigan, the current weather is as follows:
Detailed status: overcast clouds
Wind speed: 2.57 m/s, direction: 320°
Humidity: 94%
Temperature: 
  - Current: 20.04°C
  - High: 20.04°C
  - Low: 20.04°C
  - Feels like: 20.55°C
Rain: {}
Heat index: None
Cloud cover: 100%
Thought: I now know the final answer
Final Answer: In michigan, the current weather is overcast with a temperature of 20.04°C and a humidity of 94%.

> Finished chain.


{'input': 'Whats is the current weather in michigan?',
 'output': 'In michigan, the current weather is overcast with a temperature of 20.04°C and a humidity of 94%.'}

In [49]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

template = """

You are a cheerful and humarous AI assistant, you help people brighten up their mood and respond back to user query

query = {query}
"""

prompt = PromptTemplate(
    input_variables=['query'],
    template=template
)


# Create Chain
generic_llm_chain = LLMChain(
    llm = llm,
    prompt = prompt
)

# Create Tool
generic_tool = Tool(
    name = 'Generic QA',
    description='Useful for general purpose question answer',
    func = generic_llm_chain.run

)


/opt/anaconda3/envs/mytooth/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [50]:
tools = [news_tools, weather_tools, generic_tool]

In [51]:
llm = OpenAI(temperature=0)

# tools = load_tools(news_tools, llm)

agent_chain = initialize_agent(
    tools=tools, 
    llm=llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

In [52]:
agent_chain('Can you tell me a joke?')



> Entering new AgentExecutor chain...
 I should use Generic QA for this question since it is a general purpose question.
Action: Generic QA
Action Input: "Can you tell me a joke?"
Observation: 
Sure! Why couldn't the bicycle stand up by itself? Because it was two-tired!
Thought: I now know the final answer
Final Answer: Sure! Why couldn't the bicycle stand up by itself? Because it was two-tired!

> Finished chain.


{'input': 'Can you tell me a joke?',
 'output': "Sure! Why couldn't the bicycle stand up by itself? Because it was two-tired!"}

In [53]:
agent_chain('What is happening in India?')



> Entering new AgentExecutor chain...
 I should use the news-agent tool to get the latest news.
Action: news-agent
Action Input: "India"
Observation: [{'title': 'Indian Kashmir to Hold Legislative Election for First Time in a Decade', 'link': 'https://www.nytimes.com/2024/08/16/world/asia/kashmir-election.html', 'source': 'The New York Times', 'date': '23 hours ago', 'thumbnail': 'https://serpapi.com/searches/66c0c6f50a545a0586a9f2bd/images/1e8734e4dd2c3046d0f854be7540bc552cd6f5a5192e352e.jpeg'}, {'title': "Controversy erupts over Hindu temple float at New York's India Day parade", 'link': 'https://www.reuters.com/world/controversy-erupts-over-hindu-temple-float-new-yorks-india-day-parade-2024-08-16/', 'source': 'Reuters', 'date': '18 hours ago', 'thumbnail': 'https://serpapi.com/searches/66c0c6f50a545a0586a9f2bd/images/1e8734e4dd2c30469a1b747bf04088dd708ef71899952df5.jpeg'}, {'title': 'Foxconn chief defends India hiring after report that it rejects married women', 'link': 'https://w

{'input': 'What is happening in India?',
 'output': "There are various events happening in India, including a legislative election in Kashmir, controversy over a Hindu temple float at a parade in New York, a defense of hiring practices by Foxconn's chief, and a national strike over a doctor's rape and murder."}